Whisper Models
- Tiny
- Small
- Medium
- Large
- Large-v2

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jul  1 14:02:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install -U datasets huggingface_hub fsspec evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.4/515.4 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.33.0
    Uninstalling huggingface-hub-0.33.0:
      Successfully uninstalled huggingface-hub-0.33.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency 

In [3]:
from datasets import Audio, load_dataset
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import torch
from evaluate import load

In [4]:
processor = WhisperProcessor.from_pretrained("openai/whisper-medium")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium").to("cuda")
forced_decoder_ids = processor.get_decoder_prompt_ids(language="dutch", task="transcribe")

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

In [20]:
data = load_dataset("bchiusano/AsymmetriesCHILDES", 'correctPatternsSix')
data_sampled = data['train'].cast_column("audio", Audio(sampling_rate=16_000))
print(data)

DatasetDict({
    train: Dataset({
        features: ['audio', 'file_name', 'transcript'],
        num_rows: 528
    })
})


In [21]:
data_sampled[0]

{'audio': {'path': 'audio_c20_740_3620.mp3',
  'array': array([-0.00237633,  0.02671747,  0.03978223, ..., -0.00588393,
         -0.00784106, -0.01021122]),
  'sampling_rate': 16000},
 'file_name': 'c20.mp3',
 'transcript': 'een meisje die gaat met de gieter '}

In [22]:
def prepare_dataset(batch):
  if batch['transcript'] != 'hij wilde er niet meer vanaf ':
    audio = batch["audio"]
    batch = processor(audio["array"], sampling_rate=audio["sampling_rate"], text=batch["transcript"])

    batch["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    return batch

In [23]:
data_sampled = data_sampled.map(prepare_dataset)

Map:   0%|          | 0/528 [00:00<?, ? examples/s]

In [24]:
print(data_sampled)

Dataset({
    features: ['audio', 'file_name', 'transcript', 'input_features', 'labels', 'input_length'],
    num_rows: 527
})


In [25]:
MIN_DURATION_IN_SECONDS = 3.0

def is_audio_length_in_range(input_length):
    return input_length > MIN_DURATION_IN_SECONDS

In [26]:
data_sampled = data_sampled.filter(is_audio_length_in_range, input_columns=["input_length"])


Filter:   0%|          | 0/527 [00:00<?, ? examples/s]

In [27]:
data_sampled

Dataset({
    features: ['audio', 'file_name', 'transcript', 'input_features', 'labels', 'input_length'],
    num_rows: 344
})

In [28]:
def map_to_pred(batch):
    audio = batch["audio"]

    #input_features = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt").input_features
    inputs = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt")
    input_features = inputs.input_features
    attention_mask = inputs.get("attention_mask")
    batch["reference"] = processor.tokenizer._normalize(batch['transcript'])

    with torch.no_grad():
        predicted_ids = model.generate(input_features.to("cuda"), attention_mask=attention_mask, forced_decoder_ids=forced_decoder_ids)[0]
    transcription = processor.decode(predicted_ids)
    batch["prediction"] = processor.tokenizer._normalize(transcription)
    return batch

In [29]:
result = data_sampled.map(map_to_pred)

Map:   0%|          | 0/344 [00:00<?, ? examples/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [33]:
pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.5 MB/s eta 0:00:00


In [35]:
wer = load("wer")
print(wer.compute(references=result["reference"], predictions=result["prediction"]))

1.287952256620664
